# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [1]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine


## Obs on the messages DF
- No NaN values but some 'original' contain the string'NaN' while the corresponding 'message' contains '#Name?'. Those messages will be discarded.
- Few messages are less than 20 caracters. They do not seem usefull. Those messages will be discarded.
- Most of the messages are less than 500 lenght and seems usefull
- Few messages containes more than 10K caracters and seems usefull
- 'message' looks like a translation to english of the 'original'. We will focus on 'message' 
- 'genre' contains 3 possible values, without NaN --> news, direct, social

### message_df keeps 26235 of the 26248 raw data

In [2]:
# load messages dataset
messages_raw = pd.read_csv('messages.csv')
print(messages_raw.shape)
messages_raw.head()

(26248, 4)


,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [16]:
# Cleaning empty messages
#

condition_empty_messages = messages_raw['message']=='#NAME?'
messages_raw['message_stripped'] = messages_raw['message'].str.strip()
condition_tiny_messages = messages_raw['message_stripped'].str.len() < 20

messages_df = messages_raw[(~condition_empty_messages)&(~condition_tiny_messages)][['message_stripped', 'genre']]
messages_df.rename(columns={'message_stripped':'text'}, inplace=True)
print(messages_df.shape)
messages_df.head()

(26235, 2)


,text,genre
0,Weather update - a cold front from Cuba that c...,direct
1,Is the Hurricane over or is it not over,direct
2,Looking for someone but no name,direct
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,"says: west side of Haiti, rest of the country ...",direct


In [4]:
# 'genre' values
messages_df['genre'].value_counts()

news      13064
direct    10773
social     2398
Name: genre, dtype: int64

## Obs on the categories DF
- categories field is composed by concatenated information
- expanded the new categories_df contains 36 hot-enconding keys for the categories
- No NaN detected
### categories_df keeps the 26248 of raw data

In [5]:
# load categories dataset
categories_raw = pd.read_csv('categories.csv')
print(categories_raw.shape)
categories_raw.head()

(26248, 2)


,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [6]:
def split_categories(row):
    ret = {}
    for kv in row.categories.split(';'):
        k, v = kv.split('-')
        ret[k]= v
    return pd.Series(ret)

categories_df = categories_raw.apply(split_categories, axis=1)
print(categories_df.shape)
categories_df.head()

(26248, 36)


,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,1,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
categories_df.dtypes[:3]

related    object
request    object
offer      object
dtype: object

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [8]:
for column in categories_df:
    # convert column from string to numeric
    categories_df[column] = categories_df[column].astype('int32') 
categories_df.dtypes[:4]

related        int32
request        int32
offer          int32
aid_related    int32
dtype: object

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [9]:
# merge datasets
df = messages_df.merge(categories_df,left_index=True, right_index=True)
print(df.shape)
df.head()

(26235, 38)


,text,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [10]:
print('Current df rows = ', df.shape[0])

# check number of duplicates
cond_duplicated_rows = df['text'].duplicated()
print('duplicated rows to remove = ', cond_duplicated_rows.sum())

# drop duplicates
df_uniques = df[~cond_duplicated_rows]
print('Current df_uniques rows = ', df_uniques.shape[0])

# check number of duplicates
nb_duplicated_texts = df_uniques['text'].duplicated().sum()
print('nb_duplicated_texts = ',nb_duplicated_texts)

Current df rows =  26235
duplicated rows to remove =  74
Current df_uniques rows =  26161
nb_duplicated_texts =  0


In [11]:
# drop duplicates
df_uniques = df[~cond_duplicated_rows]
print('Current df_uniques rows = ', df_uniques.shape[0])

Current df_uniques rows =  26161


In [12]:
# check number of duplicates
nb_duplicated_texts = df_uniques['text'].duplicated().sum()
print('nb_duplicated_texts = ',nb_duplicated_texts)

nb_duplicated_texts =  0


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///ud-ds-disaster-message.db')
df.to_sql('disasters_messages', engine, index=False)

In [ ]:
print (engine.table_names())

from sqlalchemy import text
with engine.connect() as conn:
    result = conn.execute(text("select * from disasters_messages"))
    row = result.fetchone()

    print(row)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.